In [1]:
import glob
import json
import logging
import re

import numpy as np
from bs4 import BeautifulSoup
from unidecode import unidecode

from utils.parse_hansard import (parse_div_left, parse_hcSpeech,
                           parse_takesSectionVOList, remove_punc, remove_punc2, convert_unicode, preprocess_htmlFullContent)

from tqdm.auto import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
DEBUG = False

In [4]:
logger = logging.getLogger('extract-speech')
if DEBUG:
    logger.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(message)s")
ch = logging.StreamHandler()
ch.setFormatter(formatter)
logger.addHandler(ch)

# Testing on samples 

Currently there are 3 different formats for hansard and hence 3 different parsers for it. 

* parse_takeSectionVOList: this should work for the more recent hansard format. Hansard json will contain the field "takeSectionVOList"
* parse_hcSpeech: For hansard json containing the field "htmlFullContent" with the class hcSpeech
* parse_div_left: For hansard json containing the field "htmlFullContent" without the class hcSpeech

Added {L} for laughter and {Clap} for applause. Can remove them for training of LM.

## htmlFullContent with hcSpeech

In [5]:
filename = "docs/sitting_2011-03-04.json"
with open(filename, 'r') as f:
    data = json.load(f)
hansard_text_list = parse_hcSpeech(data, [], filename)

In [6]:
hansard_text_list

[]

## takesSectionVOList

Current version of hansard can be parsed by this. 
This also extracts the speaker info as well as topics and subtopics.

In [7]:
'<p> sdf'.split('<p>')

['', ' sdf']

In [8]:
filename = "docs/sitting_2021-04-05.json"
with open(filename, 'r') as f:
    data = json.load(f)
hansard_info_list = parse_takesSectionVOList(data, [], filename)

## Another htmlFullContent

For this case there is no detection of change in language.

In [9]:
filename = "docs/sitting_2010-05-18.json"
with open(filename, 'r') as f:
    data = json.load(f)

In [10]:
hansard_text_list = parse_div_left(data, [], filename)

# Running on all files

## Check format 

In [11]:
# Checking if the json is a seen format
count = 0
for file in glob.glob('docs/*.json'):
    with open(file, 'r') as f:
        data = json.load(f)
    if "takesSectionVOList" in data:
        for article in data['takesSectionVOList']:
            if 'content' not in article:
                logger.warning("No content %s" % file)
    elif "htmlFullContent" in data:
        html = data["htmlFullContent"].replace("\r\n", " ")
        soup = BeautifulSoup(html, 'html.parser')
        if not soup.find_all(class_="hcSpeech"):
            logger.debug("No hcSpeech %s" % file)
        else:
            count += 1
    else:
        logger.warning("Unknown Format %s" % file)

## Runs 

In [12]:
hansard_info_list = []
do_hcspeech = True
do_takesSection = True
do_divleft = True
for filename in glob.glob('docs/*.json'):
    with open(filename, 'r') as f:
        data = json.load(f)
    if "takesSectionVOList" in data and do_takesSection:
        print(filename)
        hansard_info_list = parse_takesSectionVOList(data, hansard_info_list, filename)
    elif "htmlFullContent" in data:
        print(filename)
        html = data["htmlFullContent"].replace("\r\n", " ")
        soup = BeautifulSoup(html, 'html.parser')
        if soup.find_all(class_="hcSpeech") and do_hcspeech:
            hansard_info_list = parse_hcSpeech(data, hansard_info_list, filename)
        elif do_divleft:
            hansard_info_list = parse_div_left(data, hansard_info_list, filename)
    else:
        print("None of the above! %s", filename)

docs/sitting_2008-10-20.json
docs/sitting_2014-11-04.json
docs/sitting_2023-04-20.json
docs/sitting_2011-10-17.json
docs/sitting_2023-02-22.json


PAGE <p>Page: 946</p> 
 Text: Page: 946
Skipping it
PAGE <p>Page: 955</p> 
 Text: Page: 955
Skipping it
PAGE <p>Page: 956</p> 
 Text: Page: 956
Skipping it
PAGE <p>Page: 957</p> 
 Text: Page: 957
Skipping it
PAGE <p>Page: 958</p> 
 Text: Page: 958
Skipping it
PAGE <p>Page: 959</p> 
 Text: Page: 959
Skipping it
PAGE <p>Page: 965</p> 
 Text: Page: 965
Skipping it
PAGE <p>Page: 966</p> 
 Text: Page: 966
Skipping it
PAGE <p>Page: 967</p> 
 Text: Page: 967
Skipping it
PAGE <p>Page: 968</p> 
 Text: Page: 968
Skipping it
PAGE <p>Page: 978</p> 
 Text: Page: 978
Skipping it
PAGE <p>Page: 979</p> 
 Text: Page: 979
Skipping it
PAGE <p>Page: 980</p> 
 Text: Page: 980
Skipping it
PAGE <p>Page: 981</p> 
 Text: Page: 981
Skipping it
PAGE <p>Page: 982</p> 
 Text: Page: 982
Skipping it
PAGE <p>Page: 983</p> 
 Text: Page: 983
Skipping it
PAGE <p>Page: 984</p> 
 Text: Page: 984
Skipping it
PAGE <p>Page: 985</p> 
 Text: Page: 985
Skipping it
PAGE <p>Page: 986</p> 
 Text: Page: 986
Skipping it
PAGE <p>Page

docs/sitting_2012-10-16.json


PAGE <p>Page: 1004</p> 
 Text: Page: 1004
Skipping it
PAGE <p>Page: 1005</p> 
 Text: Page: 1005
Skipping it
PAGE <p>Page: 1009</p> 
 Text: Page: 1009
Skipping it
PAGE <p>Page: 1010</p> 
 Text: Page: 1010
Skipping it
PAGE <p>Page: 1011</p> 
 Text: Page: 1011
Skipping it
PAGE <p>Page: 1011</p> 
 Text: Page: 1011
Skipping it
PAGE <p>Page: 8</p> 
 Text: Page: 8
Skipping it
PAGE <p>Page: 16</p> 
 Text: Page: 16
Skipping it


docs/sitting_2014-03-03.json


...

docs/sitting_2016-11-10.json
docs/sitting_2022-03-03.json


PAGE <p>Page: 1192</p> 
 Text: Page: 1192
Skipping it
PAGE <p>Page: 1211</p> 
 Text: Page: 1211
Skipping it
PAGE <p>Page: 1251</p> 
 Text: Page: 1251
Skipping it


docs/sitting_2012-11-14.json


PAGE <p>Page: 1261</p> 
 Text: Page: 1261
Skipping it
PAGE <p>Page: 1262</p> 
 Text: Page: 1262
Skipping it
PAGE <p>Page: 1263</p> 
 Text: Page: 1263
Skipping it
PAGE <p>Page: 1264</p> 
 Text: Page: 1264
Skipping it
PAGE <p>Page: 1265</p> 
 Text: Page: 1265
Skipping it
PAGE <p>Page: 1266</p> 
 Text: Page: 1266
Skipping it
PAGE <p>Page: 1267</p> 
 Text: Page: 1267
Skipping it
PAGE <p>Page: 1268</p> 
 Text: Page: 1268
Skipping it
PAGE <p>Page: 1269</p> 
 Text: Page: 1269
Skipping it
PAGE <p>Page: 1270</p> 
 Text: Page: 1270
Skipping it
PAGE <p>Page: 1271</p> 
 Text: Page: 1271
Skipping it
PAGE <p>Page: 1274</p> 
 Text: Page: 1274
Skipping it
PAGE <p>Page: 1275</p> 
 Text: Page: 1275
Skipping it
PAGE <p>Page: 1276</p> 
 Text: Page: 1276
Skipping it
PAGE <p>Page: 1277</p> 
 Text: Page: 1277
Skipping it
PAGE <p>Page: 1278</p> 
 Text: Page: 1278
Skipping it
PAGE <p>Page: 1279</p> 
 Text: Page: 1279
Skipping it
PAGE <p>Page: 1279</p> 
 Text: Page: 1279
Skipping it
PAGE <p>Page: 1295</p> 
 Tex

docs/sitting_2015-01-30.json
docs/sitting_2006-11-02.json
docs/sitting_2023-03-20.json


In [13]:
len(hansard_info_list)

474623

In [14]:
import os
dialogue_by_sitting = {}
for hil in tqdm(hansard_info_list):
    fp = hil.filename
    fn = fp.split(os.path.sep)[-1]
    if fn not in dialogue_by_sitting:
        f_speeches = []
    else:
        f_speeches = dialogue_by_sitting[fn]
        
    f_speeches.append(hil)
    
    dialogue_by_sitting[fn] = f_speeches

  0%|          | 0/474623 [00:00<?, ?it/s]

In [15]:
dialogue_by_sitting['sitting_2007-10-23.json'][0]

HansardInfo(speaker='The Minister of State for Trade and Industry (Mr Lee Yi Shyan) (for the Minister for Trade and Industry)', filename='docs/sitting_2007-10-23.json', title=None, subtitle=None, topic=None, text='Mr Speaker, Sir, for the second year running, the World Bank has ranked Singapore as the easiest place in the world to do business, among 178 countries. Compared to last year, we have improved or maintained our ranking in all of the 10 areas covered. This ranking speaks well of our pro-enterprise and business-friendly environment.')

In [16]:
import json
for fn, dialogues in dialogue_by_sitting.items():    
    with open('./sessions_by_sitting/' + fn[:-4] + 'jsonl', 'w') as f:
        for d in dialogues:
            f.write(json.dumps({
                'speaker': d.speaker,
                'title': d.title,
                'subtitle': d.subtitle,
                'topic': d.topic,
                'text': d.text
                }))
            f.write('\n')         

In [17]:
# import json
# for fn, dialogues in dialogue_by_sitting.items():    
#     with open('./sessions_by_sitting_merge_speakers/' + fn[:-4] + 'jsonl', 'w') as f:
#         speaker = None
#         speaker_text = []
#         title = None
#         subtitle = None
#         topic = None
#         for d in dialogues:
#             if d.speaker == speaker:
#                 speaker_text.append(d.text)
#             else:
#                 if speaker is not None:                
#                     f.write(json.dumps({
#                         'speaker': speaker,
#                         'title': title,
#                         'subtitle': subtitle,
#                         'topic': topic,
#                         'text': '\n'.join(speaker_text)
#                         }))
#                     f.write('\n')
                
#                 speaker = d.speaker
#                 speaker_text = []
#                 title = None
#                 subtitle = None
#                 topic = None
                
#         # write last segment
#         if speaker is not None:                
#                     f.write(json.dumps({
#                         'speaker': speaker,
#                         'title': title,
#                         'subtitle': subtitle,
#                         'topic': topic,
#                         'text': '\n'.join(speaker_text)
#                         }))
#                     f.write('\n')
        

In [18]:
with open('hansard.txt', 'w') as f:
    for info in hansard_info_list:
        f.writelines(info.text+"\n")

To get better speaker label when aligning the text, removing adjacent near duplicates if they are spoken by different people. For pure text alignment maybe can keep them assuming the groundtruth is really accurate.

In [19]:
from utils.align import edit_dist2
old_hansard_info=hansard_info_list[0]
slim_hansard_text_list = []
for hansard_info in hansard_info_list:
    if hansard_info.text.split():
        # Allowing dup or near dup is they are the same speaker
        if old_hansard_info.speaker == hansard_info.speaker:
            slim_hansard_text_list.append(hansard_info)
        elif not old_hansard_info.text == hansard_info.text:
            sim = edit_dist2(old_hansard_info.text.split(), hansard_info.text.split(), similarity_fun=lambda x,y: 1 if x == y else 0)
            if sim/len(hansard_info.text.split()) > 0.9:
                print(hansard_info.filename)
                print(old_hansard_info.speaker)
                print(old_hansard_info.text)
                print(hansard_info.speaker)
                print(hansard_info.text)
                print()
                slim_hansard_text_list.pop(-1)
            elif sim/len(old_hansard_info.text.split()) > 0.9:
                print(hansard_info.filename)
                print("reverse")
                print(old_hansard_info.speaker)
                print(old_hansard_info.text)
                print(hansard_info.speaker)
                print(hansard_info.text)
                print()
                slim_hansard_text_list.pop(-1)
            else:
                slim_hansard_text_list.append(hansard_info)
                old_hansard_info=hansard_info
        else: #Drop same words but different speakers
            print("same!")
            print(hansard_info.filename)
            print(hansard_info.text)
            print()

ModuleNotFoundError: No module named 'utils.align'

In [ ]:
len(slim_hansard_text_list)

In [ ]:
with open('hansard_speech.txt', 'w') as f:
    for sent in slim_hansard_text_list:
        f.writelines(sent+"\n")